# Lookout Device Compliance Threat Hunting

This notebook provides threat hunting queries for investigating device compliance and security posture issues.

## Prerequisites
- Lookout data connector configured in Microsoft Sentinel
- LookoutEvents parser deployed
- DEVICE events ingested

## 1. Non-Compliant Devices Overview

Identify devices that are not meeting compliance requirements.

In [ ]:
LookoutEvents
| where EventType == "DEVICE"
| where DeviceComplianceStatus in ("Non-Compliant", "Pending")
| summarize arg_max(TimeGenerated, *) by DeviceGuid
| project 
    DeviceEmailAddress,
    DevicePlatform,
    DeviceManufacturer,
    DeviceModel,
    DeviceOSVersion,
    DeviceActivationStatus,
    DeviceComplianceStatus,
    LastSeen = TimeGenerated
| sort by LastSeen desc

## 2. Outdated Operating Systems

Find devices running potentially vulnerable OS versions.

In [ ]:
LookoutEvents
| where EventType == "DEVICE"
| summarize arg_max(TimeGenerated, *) by DeviceGuid
| extend OSMajorVersion = toint(split(DeviceOSVersion, ".")[0])
| extend IsOutdated = case(
    DevicePlatform == "ANDROID" and OSMajorVersion < 12, true,
    DevicePlatform == "IOS" and OSMajorVersion < 15, true,
    false
)
| where IsOutdated == true
| summarize DeviceCount = count() by DevicePlatform, DeviceOSVersion
| sort by DeviceCount desc

## 3. Device Security Status Distribution

Understand the security posture across your mobile fleet.

In [ ]:
LookoutEvents
| where EventType == "DEVICE"
| summarize arg_max(TimeGenerated, *) by DeviceGuid
| summarize Count = count() by DeviceSecurityStatus, DevicePlatform
| sort by Count desc

## 4. Inactive Devices

Identify devices that haven't checked in recently.

In [ ]:
LookoutEvents
| where EventType == "DEVICE"
| where DeviceActivationStatus == "INACTIVE"
| summarize arg_max(TimeGenerated, *) by DeviceGuid
| project 
    DeviceEmailAddress,
    DevicePlatform,
    DeviceManufacturer,
    DeviceModel,
    DeviceActivationStatus,
    LastSeen = TimeGenerated
| sort by LastSeen asc

## 5. Device Platform Distribution

Overview of mobile device platforms in your environment.

In [ ]:
LookoutEvents
| where EventType == "DEVICE"
| summarize arg_max(TimeGenerated, *) by DeviceGuid
| summarize Count = count() by DevicePlatform, DeviceManufacturer
| sort by Count desc
| render piechart